# Cómo construir y desplegar un modelo de aprendizaje automático con FastAPI

Crear un modelo de aprendizaje automático es solo una parte del proyecto. Para que sea útil en el mundo real, debe estar accesible para los usuarios y desarrolladores. El método más fácil y más utilizado para desplegar modelos de aprendizaje automático es envolverlos dentro de una API REST. Eso es precisamente lo que haremos hoy, con una biblioteca en tendencia: FastAPI.

FastAPI es una biblioteca moderna y rápida para construir APIs con Python 3.6+. En términos de rendimiento, está al mismo nivel que NodeJS y Go. También es fácil de aprender y viene con documentación interactiva automática.

Este notebook pretende cubrir lo suficiente de la biblioteca para que puedas empezar a desplegar modelos. Al final de la lectura, sabrás cómo desplegar un modelo de aprendizaje automático y usarlo para hacer predicciones ya sea desde Python, la línea de comandos, u otros lenguajes de programación.

El notebook se estructura de la siguiente manera:

- Instalación de FastAPI y construcción de la primera API
- Exploración de la documentación interactiva
- Entrenamiento de un modelo de aprendizaje automático
- Construcción de una API REST completa
- Pruebas
- Conclusión

## Instalación FastAPI

Primero, tendrás que instalar la biblioteca junto con un servidor ASGI - tanto Uvicorn como Hypercorn están bien. Ejecutar esta línea desde la Terminal:

> pip install fastapi uvicorn

A continuación, puedes crear una carpeta para la API y abrirla en tu editor de código favorito. Para continuar, crea un script de Python llamado app.py. Aquí hay una lista de cosas que tienes que hacer para hacer una API simple con dos endpoints:

- Importar las bibliotecas: FastAPI y Uvicorn
- Crear una instancia de la clase FastAPI
- Declarar la primera ruta: devuelve un simple objeto JSON en la página de índice (http://127.0.0.1:8000)
- Declarar la segunda ruta: devuelve un simple objeto JSON que contiene un mensaje personalizado. El parámetro name viene directamente de la URL (por ejemplo, http://127.0.0.1:8000/John)
- Ejecutar la API con Uvicorn

El siguiente fragmento de código muestra cómo implementar estos cinco pasos:

In [1]:
# 1. Importación de las bibliotecas
import uvicorn
from fastapi import FastAPI

# 2. Crear el objeto de la aplicación
app = FastAPI()

# 3. Ruta del índice, se abre automáticamente en http://127.0.0.1:8000
@app.get('/')
def index():
    '''
    Este es un docstring...
    '''
    return {'message': 'Hola, desconocido'}

# 4. Ruta con un único parámetro, devuelve el parámetro dentro de un mensaje
#    Ubicado en: http://127.0.0.1:8000/CualquierNombre
@app.get('/{name}')
def get_name(name: str):
    '''
    Este es el otro docstring...
    '''
    return {'message': f'Hola, {name}'}

# 5. Ejecutar la API con uvicorn
#    Se ejecutará en http://127.0.0.1:8000
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop

Luego, para ejecutar nuestra API, abra una ventana de Terminal donde se encuentre app.py. Ahora escribe lo siguiente:

> uvicorn app:app --reload

## Documentación Interactiva

FastAPI también proporciona documentación interactiva para nuestra API. Puedes ver esta documentación en el siguiente enlace: http://127.0.0.1:8000/docs

Vamos a probarla..

## Entrenando un modelo de ML

A continuación, vamos a entrenar un modelo de aprendizaje automático. Para este ejemplo, vamos a utilizar el conjunto de datos Iris y el algoritmo de Random Forest.

Cuales serian los pasos a seguir?

- Importaciones: necesitarás pandas, RandomForestClassifier de scikit-learn, BaseModel de pydantic (verás por qué en el próximo paso) y joblib, para guardar y cargar modelos.
- Declara una clase IrisSpecies que herede de BaseModel. Esta clase contiene solo campos que se utilizan para predecir una sola especie de flor (más sobre esto en la próxima sección).
- Declara una clase IrisModel, utilizada para entrenar el modelo y hacer predicciones.
- Dentro de IrisModel, declara un método llamado _train_model. Se utiliza para realizar el entrenamiento del modelo con el algoritmo de Random Forests. El método devuelve el modelo entrenado.
- Dentro de IrisModel, declara un método llamado predict_species. Se utiliza para hacer una predicción a partir de 4 parámetros de entrada (medidas de la flor). El método devuelve la predicción (especie de flor).
- Dentro de IrisModel, modifica el constructor, para que cargue el conjunto de datos Iris y entrene el modelo si este no existe en la carpeta. Esto aborda el problema de entrenar un nuevo modelo cada vez. La biblioteca joblib se utiliza para guardar y cargar modelos.

 Crea un archivo llamado Model.py e inserta el siguiente código:

In [ ]:
# 1. Importación de bibliotecas
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from pydantic import BaseModel
import joblib

# 2. Clase que describe las medidas de una única flor
class IrisSpecies(BaseModel):
    sepal_length: float 
    sepal_width: float 
    petal_length: float 
    petal_width: float

# 3. Clase para entrenar el modelo y hacer predicciones
class IrisModel:
    # Constructor de la clase, carga el conjunto de datos y carga el modelo
    # si existe. Si no, llama al método _train_model y 
    # guarda el modelo
    def __init__(self):
        self.df = pd.read_csv('iris.csv')
        self.model_fname_ = 'iris_model.pkl'
        try:
            self.model = joblib.load(self.model_fname_)
        except Exception as _:
            self.model = self._train_model()
            joblib.dump(self.model, self.model_fname_)

    # 4. Entrena el modelo usando el clasificador RandomForest
    def _train_model(self):
        X = self.df.drop('variety', axis=1)
        y = self.df['variety']
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
        model = RandomForestClassifier(n_estimators=100)
        model.fit(X_train, y_train)
        return model

    # 5. Realiza una predicción utilizando el modelo entrenado
    def predict_species(self, sepal_length, sepal_width, petal_length, petal_width):
        data_in = [[sepal_length, sepal_width, petal_length, petal_width]]
        prediction = self.model.predict(data_in).tolist()
        probabitlity = self.model.predict_proba(data_in).tolist()
        return prediction, probabitlity


## Creación de una API REST completa

Volvamos al archivo app.py y bórralo todo. Deberíamos empezar de nuevo con un archivo en blanco, aunque la estructura básica será más o menos idéntica a la que tenías antes.

Esta vez, declararás solo un punto final, utilizado para predecir la especie de la flor. Este punto final realiza la predicción llamando al método IrisModel.predict_species(), declarado en la sección anterior. El otro cambio significativo es el tipo de solicitud. POST es lo que quieres con las APIs de aprendizaje automático, ya que se considera una mejor práctica enviar parámetros en JSON en lugar de en URL.

La lista de tareas para app.py es bastante corta:

- Importaciones: necesitarás uvicorn, FastAPI, IrisModel e IrisSpecies del archivo Model.py previamente escrito.
- Haz una instancia de FastAPI y de IrisModel.
- Declara una función para hacer predicciones, ubicada en http://127.0.0.1:8000/predict. La función toma un objeto de tipo IrisSpecies, lo convierte en un diccionario y lo pasa al método IrisModel.predict_species(). Se devuelve la clase predicha y la probabilidad predicha.
- Ejecuta la API utilizando uvicorn.

Debería quedar algo asi:

In [ ]:
# 1. Importación de bibliotecas

import uvicorn
from fastapi import FastAPI
from Model import IrisModel, IrisSpecies

# 2. Crear la aplicación y los objetos del modelo

app = FastAPI()
model = IrisModel()

# 3. Exponer la funcionalidad de predicción, realizar una predicción a partir de los datos JSON pasados
#    y devolver la especie de flor predicha con la confianza

@app.post('/predict')
def predict_species(iris: IrisSpecies):
    data = iris.dict()
    prediction, probability = model.predict_species(
        data['sepal_length'], data['sepal_width'], data['petal_length'], data['petal_width']
    )
    return {
        'prediction': prediction,
        'probability': probability
    }

# 4. Ejecutar la API con uvicorn
#    Se ejecutará en http://127.0.0.1:8000

if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)

## Test

Para ejecutar la API, una vez más, introduce el siguiente texto en la Terminal: 

> uvicorn app:app -–reload

Ahora podemos ver todo en la interfaz de documentación

O incluso a través de cualquier lenguaje de programación (ejemplo en Python):

In [1]:
import requests 

new_measurement = {
    'sepal_length': 5.7,
    'sepal_width': 3.1,
    'petal_length': 4.9,
    'petal_width': 2.2
}

response = requests.post('http://127.0.0.1:8000/predict', json=new_measurement)
print(response.content)

b'{"prediction":["Virginica"],"probability":[[0.0,0.2,0.8]]}'
